In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
soup = []
for page in range(0, 366 + 1):  # 366 페이지까지 
    URL = 'https://www.safetykorea.kr/recall/recallBoard'
    response = requests.post(URL, data={
        'selectedOption': 0,
        'pageNo': f'{page}',
    })
    soup.append(BeautifulSoup(response.text, 'html.parser'))

In [ ]:
recalls = []
for data in soup:
    datas = data.select('td', class_='ta_center')
    for i in range(len(datas)):
        if i % 8 != 0:
            continue
        recalls.append({
            'ID': datas[i].text, 
            '공표일': datas[i + 7].text,
            '기업명': datas[i + 4].text, 
            '품목': datas[i + 2].text.strip(), 
            '리콜종류': datas[i + 5].text, 
        })

columns = ','.join(recalls[0].keys())
# print(columns)
columns = ['ID', '공표일', '기업명', '품목', '리콜종류']
values = []
for recall in recalls:
    values.append(tuple(recall.values()))

f = open('recall.csv', 'w', encoding='utf-8')
f.write('ID\t공표일\t기업명\t품목\t리콜종류\n')
for idx, recall in enumerate(recalls):
    f.write(recall['ID'] + '\t' + recall['공표일'] + '\t' + recall['기업명']
    + '\t' + recall['품목'] + '\t' + recall['리콜종류'] + '\n' )
    if (idx + 1) % 80 == 0:  # 페이지당 10개의 데이터, 10의 배수로 나타냄
        print(int((idx + 1) / 8), "페이지 크롤링 완료했습니다.")

f.close()

#### ----------------------------- 리콜 크롤링 후 파일로 만들었으니 여기서부터 건드시면 됩니다. -----------------------------------

In [16]:
# 리콜 기업리스트 정리
import pandas as pd

file_path = 'recall.csv'

recall_df = pd.read_csv(file_path, sep='\t')
recall_df = recall_df.drop(columns='ID') # '품목'
index_to_drop = [3668, 3669]  # 삭제할 행의 인덱스
recall_df = recall_df.drop(index_to_drop)
recall_df['기업명'] = recall_df['기업명'].str.replace("\(주\)|주식회사|㈜|주\)", "", regex=True)

recall_df['공표일'] = pd.to_datetime(recall_df['공표일'])
recall_df['공표일'] = recall_df['공표일'].dt.year
recall_df = recall_df.rename(columns={'공표일': '인증연도'})
recall_df = recall_df.rename(columns={'품목': '제품명'})
recall_df = recall_df[['기업명', '인증연도', '리콜종류', '제품명']]

# 기업명칭 통일한 리콜 기업리스트 
rename = {
    '신세계코리아|신세계인터내셔날|Shenzhen Fujia Appliance Co.,Ltd\\(신세계코리아\\)': '신세계',
    '한국타이어': '한국타이어앤테크놀로지',
    '롯데쇼핑롯데마트사업본부|롯데쇼핑GF사업본부': '롯데쇼핑',
    'LS네트웍스': 'LS',
    'SK매직': 'SK',
    '한화전기': '한화',
    'LG패션': 'LG'
}

for old_name, new_name in rename.items():
    recall_df['기업명'] = recall_df['기업명'].replace(old_name, new_name, regex=True)
    recall_df['기업명'] = recall_df['기업명'].str.strip()
rename_recall_df = recall_df
rename_recall_df

,기업명,인증연도,리콜종류,제품명
0,신지모루,2024,자발적리콜,마그네틱 보조배터리
1,휴리엔,2024,자발적리콜,가정용 음식물처리기
2,MTD Products Inc.,2024,자발적리콜,"잔디깍기(제로턴 잔디깍기, Zero Turn Lawn Mowers)"
3,산바다스포츠,2024,자발적리콜,캐논데일 데이브 자전거
4,하츠,2024,자발적리콜,전기레인지
...,...,...,...,...
3663,DAWOO LUX,2012,명령에따른리콜,형광등용 안정기
3664,동성조명,2012,명령에따른리콜,고정형형광등기구
3665,아림산업,2012,명령에따른리콜,형광등기구
3666,에프앤에프,2012,명령에따른리콜,어린이 책가방


## --------------------------------- TOP 200 KOSPI ---------------------------------

In [2]:
# top_200 기업리스트 불러오기 
file_path = 'top200_name_list.csv'
top_200 = pd.read_csv(file_path, sep='\t')

In [5]:
# # recall_df, top_200 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(recall_df['기업명']).split())
# words_df2 = set(' '.join(top_200['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'삼풍산업', '현대모비스', '케미칼', '게임즈', '라', 'ILSHIN', '동서', '신세계코리아', '더', '이노베이션', '컴퍼니', 'SKID', 'SK매직', '제일', '성림GS', 'LS네트웍스', '세모에스엘', 'TOOLS', '디스', '에코', '코웨이', '한일', '효성', '세계', '(대동하이브리드', '삼성물산', '롯데쇼핑GF사업본부', '대상엘이디', '인터내셔널', '한샘', '에이치에스엘이디라이팅', 'GB', 'LG패션', '동서가구', '트리', '대상의료기', '온', '미래', '아이', '두산테크', '뉴(NEW)대웅', 'HENGSHUN', '오리온씨엔씨', '한국타이어', '현대일렉트릭', '신세계인터내셔날', '&', '에스엘이노텍', '이마트', 'LG전자', '물산', '풍산사다리', '한화', '한화전기', '앤', '삼성전자', '금호타이어', '롯데쇼핑롯데마트사업본부', 'Co.,Ltd(신세계코리아)', '한화의료기'}


In [ ]:
# 같은 기업이라도 자료에 따라 기업명칭이 다르게 기입되어있고, 다양한 계열사로 인해 두 데이터에서 동일한 값(기업명)을 얻기 어렵다고 판단하여 직접 체크했습니다.
# top 200 list의 '기업명' 기준으로 두 데이터의 '기업명'을 통일하고자 합니다. 

In [4]:
# 리콜 기업리스트 중 top 200의 리콜 기록
recall_companies = ['현대모비스', '신세계', '한국타이어앤테크놀로지', '삼성물산', '삼성전자', 
                    '롯데쇼핑', '코웨이', 'LS', 'SK', '한샘', '한화', 'LG전자', 'LG', '효성', '이마트', '금호타이어']

top200_recall = rename_recall_df[rename_recall_df['기업명'].isin(recall_companies)]
top200_recall = top200_recall.reset_index(drop=True)
top200_recall

,기업명,인증연도,리콜종류,제품명
0,이마트,2023,명령에따른리콜,플러그 및 콘센트(전기설비용 부속품 및 연결부품)
1,SK,2022,자발적리콜,6인용식기세척기
2,삼성전자,2022,자발적리콜,전기 세탁기 (드럼세탁기)
3,삼성물산,2022,명령에따른리콜,외의류(아동용 섬유제품)
4,신세계,2021,명령에따른리콜,가정용 섬유제품
5,LG,2021,자발적리콜,모가미 샌들 4종 (Black/Blue/Lime/Pink)
6,롯데쇼핑,2021,명령에따른리콜,아동용 섬유제품
7,금호타이어,2020,명령에따른리콜,자동차용 타이어
8,이마트,2020,자발적리콜,건전지(알칼리·망간 전지)
9,LG전자,2020,자발적리콜,텔레비전수상기


In [17]:
# top200_recall.to_csv('top200_recall_list.csv', index=False)

## --------------------------------- TOP 150 KOSDAQ ---------------------------------

In [9]:
# top_150 코스닥 기업리스트 불러오기 
file_path = 'top150_kosdaq.csv'
top_150 = pd.read_csv(file_path, sep='\t')
top_150 = top_150.rename(columns={'종목명': '기업명'})

In [13]:
# # rename_recall_df, top_150 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(rename_recall_df['기업명']).split())
# words_df2 = set(' '.join(top_150['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'디어', '에스티', '더네이쳐홀딩스', '에스엠코프', '라', '&', '미래', 'HK', '프로텍메디칼', '아이', '게임즈', '디오', '에스엠어패럴', '씨케이', '이홀딩스', '에코', '더', '패션', '에스엠디네트웍스', '케이엠', '에스엠비', '앤', '온', '트리'}


In [15]:
# 리콜 기업리스트 중 top 150의 리콜 기록
recall_companies = ['더네이쳐홀딩스']

top150_recall = rename_recall_df[rename_recall_df['기업명'].isin(recall_companies)]
top150_recall = top150_recall.reset_index(drop=True)
top150_recall

,기업명,인증연도,리콜종류,제품명
0,더네이쳐홀딩스,2019,명령에따른리콜,외의류(아동용 섬유제품)
